In [2]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd




In [39]:
start = 0
begin_date = "2000-05-01"
end_date = "2000-07-01"
url = f"http://www.prosportstransactions.com/basketball/Search/SearchResults.php?Player=&Team=&BeginDate={begin_date}&EndDate={end_date}&PlayerMovementChkBx=yes&Submit=Search&start={str(start)}"
html = urlopen(url)
soup = BeautifulSoup(html)
# soup.find_all




In [40]:
transactions = []
table_rows = soup.find_all('div')[3].table.find_all('tr')
if len(table_rows) == 1:
    print("no more")
for trans in table_rows[1:]:
    trans_data = trans.find_all('td')

    date = trans_data[0].contents[0]

    team = trans_data[1].contents[0].strip()

    acquired = trans_data[2].contents[0]
    if acquired:
        acquired = acquired[3:]

    relinquished = trans_data[3].contents[0]
    if relinquished:
        relinquished = relinquished[3:]

    notes = trans_data[4].contents[0]

    transactions += (date, team, acquired, relinquished, notes)

In [120]:
def process_name(name: str):
    name = name.strip()
    if name and name[0] == "•":
        name = name[2:]
    return name

def get_player_transactions(year: int):
    begin_date = f"{str(year)}-01-01"
    end_date = f"{str(year)}-12-31"
    url = f"http://www.prosportstransactions.com/basketball/Search/SearchResults.php?Player=&Team=&BeginDate={begin_date}&EndDate={end_date}&PlayerMovementChkBx=yes&Submit=Search&start="
    start = 0
    
    transactions = []

    while True:
        # print("Start: ", start)
        current_url = url + str(start)
        start += 25

        html = urlopen(current_url)
        soup = BeautifulSoup(html)
        table_rows = soup.find_all('div')[3].table.find_all('tr')
        if len(table_rows) == 1:
            break
        
        for trans in table_rows[1:]:
            trans_data = trans.find_all('td')

            date = trans_data[0].contents[0]

            team = trans_data[1].contents[0].strip()

            if trans_data[2].contents[0]:
                acquired = list(map(process_name, trans_data[2].contents[::2]))
            else:
                acquired = []
            # if acquired:
            #     acquired = acquired[3:]

            if trans_data[3].contents[0]:
                relinquished = list(map(process_name, trans_data[3].contents[::2]))
            else:
                relinquished = []
            # if relinquished:
            #     relinquished = relinquished[3:]

            notes = trans_data[4].contents[0]

            transactions.append((date, team, acquired, relinquished, notes))
            
    return transactions



In [105]:
trans = get_player_transactions(2000)


In [106]:
trans

[('2000-10-16',
  'Cavaliers',
  ['Michael Hawkins'],
  [''],
  ' signed free agent to a $523.5K contract'),
 ('2000-10-16',
  'Celtics',
  ['Bryant Stith', 'Chris Herren'],
  ['Robert Pack', 'Calbert Cheaney'],
  ' trade with Nuggets'),
 ('2000-10-16', 'Hornets', [''], ['Caswell Cyrus'], ' waived'),
 ('2000-10-16',
  'Mavericks',
  ['Dirk Nowitzki'],
  [''],
  ' team exercised contract option through 2001-02'),
 ('2000-10-16',
  'Nuggets',
  ['Keon Clark'],
  [''],
  ' team exercised fourth year contract option'),
 ('2000-10-16',
  'Nuggets',
  ['Raef LaFrentz'],
  [''],
  ' team exercised fourth year contract option'),
 ('2000-10-16',
  'Nuggets',
  ['Robert Pack', 'Calbert Cheaney'],
  ['Bryant Stith', 'Chris Herren'],
  ' trade with Celtics'),
 ('2000-10-16', 'Suns', [''], ['Carlos Daniel'], ' waived'),
 ('2000-10-16', 'Suns', [''], ['Reed Rawlings'], ' waived'),
 ('2000-10-16', 'Warriors', [''], ['Jerod Ward'], ' waived'),
 ('2000-10-16', 'Warriors', [''], ['Rick Hughes'], ' waive

In [53]:
a = []
a.append((3, 4))
a

[(3, 4)]

In [56]:
dir = "C:/Users/bhalb/nbaproject/data/transactions1999.csv"

df= pd.DataFrame(trans, columns=["date", "team", "acquired", "relinquished", "notes"])
df.to_csv(dir, index=False)


In [111]:
def save_transaction_data(year):
    path = f"C:/Users/bhalb/nbaproject/data/transactions{str(year)}.json"
    trans = get_player_transactions(year)
    df = pd.DataFrame(
        trans, columns=["date", "team", "acquired", "relinquished", "notes"])
    df.to_json(path)


In [123]:
for i in range(2001, 2022):
    save_transaction_data(i)

In [232]:
def indicates_player_signing(note: str):

    player_signing_starts = (
        "waived",
        "signed",
        "re-signed",
        "re-resigned",
        "deceased",
        "player became a free agent",
        "contract expired",
        "claimed off waivers",
        "retired",
        "reached agreement to buy out contract",
        "player agreed to waive",
        "announced retirement",
        "player exercised",
        "team exercised",
        "player declined",
        "team declined",
        "player opted out",
        "player became",
        "lost free agent",
        "lost restricted free agent",
        "lost restricted free agnet",
        "renounced rights",
        "contract voided by NBA",
        "un-retired and signed",
        "player reneged on contract",
    )

    # not re-signed as head coach
    if note.startswith(player_signing_starts):
        return True

    return False

def indicates_ignore(note: str):
    ignore_starts = (
        "re-signed as head",
        "resigned as",
        "retired as",
        "resigned/retired as",
        "hired",
        "fired",
        "transfer of ownership",
        "purchased team",
        "purchase of team",
        "promoted to",
        "re-assigned",
        "named",
        "NBA approved purchase of team",
        "agreed to sell team",
        "agreed to purchase team",
        "reached agreement for transfer of ownership",
        "assistant coach",
        "special assistant coach",
        "team added",
        "London arbitrator",
        "exercised their option to claim",
        "Announced they will exercise the option on",
        "announced he will return as head coach",
        "ammended 8/12/01 trade",
        "awarded by NBA",
        "2000 NBA draft",
        "2001 NBA draft",
        "2002 NBA draft",
        "2003 NBA draft",
        "2004 NBA draft",
        "2005 NBA draft",
        "2006 NBA draft",
        "2007 NBA draft",
        "2008 NBA draft",
        "2009 NBA draft",
        "2010 NBA draft",
        "2011 NBA draft",
        "2012 NBA draft",
        "2013 NBA draft",
        "2014 NBA draft",
        "2015 NBA draft",
        "2016 NBA draft",
        "2017 NBA draft",
        "2018 NBA draft",
        "2019 NBA draft",
        "2020 NBA draft",
        "2021 NBA draft",
    )
    # not re-signed as head coach

    if note.startswith(ignore_starts):
        return True

    return False

def indicates_trade(note: str):
    trade_starts = (
        "trade",
        "announced trade",
        "3-team trade",
        "4-team trade",
        "5-team trade",
        "6-team trade",
        "earlier trade with",
        "sent to", # just picks?
        "received from",
    )
    # rights to
    # 2000
    if note.startswith(trade_starts):
        return True
    return False

In [233]:
# import csv

year = 2001
path = f"C:/Users/bhalb/nbaproject/data/transactions{str(year)}.json"
# with open(path, newline='') as csvfile:
#     reader = csv.reader(csvfile, 
df = pd.read_json(path)
season_start = "2000-10-31"
for row in df.itertuples():
    note = row.notes.strip()
    if indicates_ignore(note):
        continue
    if indicates_player_signing(note):
        continue
    if indicates_trade(note):
        continue
    print(note, row.acquired, row.relinquished)
    # TODO when player added to team, remove from old team
    break
    if len(row.acquired) > 1:
        
        break
    # if d.date >= season_start:
    #     if 
    #     print(d)
    #     break


player did not report to training camp [''] ['Etdrick Bohannon']


In [31]:
soup.find_all('div')[3].table.find_all('tr')[1:]



[]

In [41]:
transactions

['2000-05-01',
 'Warriors',
 '',
 'Rod Higgins',
 ' assistant coach (date approximate)',
 '2000-05-03',
 'Nets',
 '',
 'Michael Rowe',
 ' resigned as president',
 '2000-05-05',
 'Blazers',
 '',
 'Bill Musselman',
 ' assistant coach deceased',
 '2000-05-08',
 'Grizzlies',
 '',
 'Jack Nolan',
 ' fired as assistant coach',
 '2000-05-08',
 'Grizzlies',
 '',
 'Jim Boylan',
 ' fired as assistant coach',
 '2000-05-08',
 'Grizzlies',
 '',
 'Larry Riley',
 ' fired as director of player personnel',
 '2000-05-08',
 'Grizzlies',
 '',
 'Lawrence Frank',
 ' fired as assistant coach',
 '2000-05-08',
 'Grizzlies',
 '',
 'Lionel Hollins',
 ' fired interim head coach',
 '2000-05-09',
 'Grizzlies',
 '',
 'Stu Jackson',
 ' resigned as president & general manager',
 '2000-05-09',
 'Mavericks',
 'Sidney Moncrief',
 '',
 ' hired as assistant coach',
 '2000-05-09',
 'Pacers',
 '',
 'Billy Knight',
 ' resigned as seniot VP/basketball (date approximate)',
 '2000-05-10',
 'Grizzlies',
 'Billy Knight',
 '',
 ' hi

In [30]:
def getTeamRatings():
    url = 'https://www.basketball-reference.com/leagues/NBA_2020_ratings.html'
    html = urlopen(url)
    soup = BeautifulSoup(html)
    headers = [th.getText() for th in soup.findAll('tr')[1].findAll('th')]
    rows = soup.findAll('tr') # 2 is first team
    team_stats = [[i-1]+[td.getText() for td in rows[i].findAll('td')] for i in range(2,len(rows))]
    stats = pd.DataFrame(team_stats, columns = headers)
    
    return stats

stats = getTeamRatings()
stats.head(30)

,Rk,Team,Conf,Div,W,L,W/L%,MOV,ORtg,DRtg,NRtg,MOV/A,ORtg/A,DRtg/A,NRtg/A
0,1,Milwaukee Bucks,E,C,7,3,.700,8.10,111.82,104.12,7.70,11.38,115.33,104.45,10.89
1,2,Los Angeles Lakers,W,P,8,2,.800,8.30,108.18,100.17,8.01,9.97,109.35,99.77,9.58
2,3,Boston Celtics,E,A,8,1,.889,9.44,113.02,103.69,9.33,8.81,112.90,104.27,8.62
3,4,Los Angeles Clippers,W,P,7,3,.700,5.00,109.99,104.94,5.04,8.08,112.13,104.19,7.94
4,5,Utah Jazz,W,NW,8,3,.727,5.00,105.49,100.56,4.93,7.45,105.59,98.30,7.29
5,6,Phoenix Suns,W,P,6,4,.600,6.20,111.44,105.79,5.65,7.64,112.49,105.45,7.04
6,7,Toronto Raptors,E,A,7,3,.700,5.80,108.43,103.05,5.38,7.03,108.50,101.86,6.65
7,8,Miami Heat,E,SE,7,3,.700,6.00,107.94,102.50,5.44,7.03,108.15,101.74,6.40
8,9,Philadelphia 76ers,E,A,7,3,.700,4.40,107.10,102.91,4.19,5.33,107.98,102.90,5.08
9,10,Denver Nuggets,W,NW,7,3,.700,2.30,106.68,104.27,2.41,2.67,106.79,104.06,2.73


In [32]:
# get lineup data
url = 'https://stats.nba.com/lineups/advanced/?Season=2018-19&SeasonType=Regular%20Season'
html = urlopen(url)
soup = BeautifulSoup(html)
#headers = [th.getText() for th in soup.findAll('tr')[1].findAll('th')]
soup

<!DOCTYPE html>
<html lang="en" ng-app="stats" ng-strict-di="" stats-build-date="2019-11-08 03:13 PM" stats-version="4.22.3" xmlns:ng="http://angularjs.org">
<head>
<meta charset="utf-8"/>
<meta content="IE=Edge,chrome=1" http-equiv="X-UA-Compatible"/><script type="text/javascript">(window.NREUM||(NREUM={})).loader_config={xpid:"VQECWF5UChAHUlNTBwgBVw==",licenseKey:"09f0cb5c68",applicationID:"76210961"};window.NREUM||(NREUM={}),__nr_require=function(t,n,e){function r(e){if(!n[e]){var o=n[e]={exports:{}};t[e][0].call(o.exports,function(n){var o=t[e][1][n];return r(o||n)},o,o.exports)}return n[e].exports}if("function"==typeof __nr_require)return __nr_require;for(var o=0;o<e.length;o++)r(e[o]);return r}({1:[function(t,n,e){function r(t){try{s.console&&console.log(t)}catch(n){}}var o,i=t("ee"),a=t(18),s={};try{o=localStorage.getItem("__nr_flags").split(","),console&&"function"==typeof console.log&&(s.console=!0,o.indexOf("dev")!==-1&&(s.dev=!0),o.indexOf("nr_dev")!==-1&&(s.nrDev=!0))}catch